In [1]:
from gurobipy import GRB
import gurobipy as gb

# Create the optimization model
model = gb.Model("LG Example")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [2]:
# Create the three classes of decision variables (type A, B, and C)
a = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Type A")
b = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Type B")
c = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Type C")

# The objective function
model.setObjective(60*a + 75*b + 80*c, GRB.MAXIMIZE)

# Add the constraints
model.addConstr(2*a + 1.5*b + 3*c <= 10000, "Assembly Constraint")
model.addConstr(a + 2*b + c <= 5000, "Package Constraint")

# We could also define these constraints as upper bounds in the definition of the decision variables
model.addConstr(a <= 3000, "Order Limit Constraint (Type A)")
model.addConstr(b <= 2000, "Order Limit Constraint (Type B)")
model.addConstr(c <= 900, "Order Limit Constraint (Type C)")

<gurobi.Constr *Awaiting Model Update*>

In [3]:
# Optimally solve the problem
model.optimize()

# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

# The status of the model (Optimization Status Codes)
print("Model Status: ", model.status)

# Print the objective
print(model.ObjVal)

# Print the decision variables
print(model.printAttr('X'))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 22.4.0 22E261)

CPU model: Intel(R) Core(TM) i5-1030NG7 CPU @ 1.10GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 3 columns and 9 nonzeros
Model fingerprint: 0x4d176d43
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [6e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+02, 1e+04]
Presolve removed 3 rows and 0 columns
Presolve time: 0.02s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+05   2.208333e+03   0.000000e+00      0s
       3    2.9325000e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.04 seconds (0.00 work units)
Optimal objective  2.932500000e+05
Number of Decision Variables:  3
Model Status:  2
293250.0

    Variable            X 
-------------------------
      Type A         3000 
      Type 

### part2

In [4]:
# Create the optimization model
model = gb.Model("Transportation Problem")

# A list of list of costs
costs = [[21, 50, 40], [35, 30, 22], [55, 20, 25]]
demand = [200000, 600000, 225000]
supply = [275000, 400000, 300000]

# Create the a single class of decision variables
x = model.addVars(3, 3, lb=0, vtype=GRB.CONTINUOUS, name="Transportation Plan")

# The objective function
model.setObjective(gb.quicksum(costs[i][j]*x[i,j] for i in range(3) for j in range(3)), GRB.MINIMIZE)

# Add the supply constraints
for i in range(3):
    model.addConstr(gb.quicksum(x[i,j] for j in range(3)) == supply[i], name="Supply Constraint %i" %i)

# Add the demand constraints
for j in range(3):
    model.addConstr(gb.quicksum(x[i,j] for i in range(3)) <= demand[j], name="Demand Constraint %i" %j)

# Optimally solve the problem
model.optimize()

# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

# Value of the objective function
print("Total Transportation cost: ", model.objVal)

# Print the decision variables
print(model.printAttr('X'))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 22.4.0 22E261)

CPU model: Intel(R) Core(TM) i5-1030NG7 CPU @ 1.10GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0x43d91904
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+05, 6e+05]
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0575000e+07   2.500000e+05   0.000000e+00      0s
       2    2.4000000e+07   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.400000000e+07
Number of Decision Variables:  9
Total Transportation cost:  24000000.0

    Variable            X 
-------------------------
Transportation Plan[0,0]       200000 
Transportation Plan[0,

### Transshipment

In [5]:
from gurobipy import GRB
import gurobipy as gb

# Create the optimization model
model = gb.Model("Transshipment Problem")

# A list of list of costs
source_costs = [[11, 10, 26, 29], [9, 12, 27, 26]]
trans_costs = [[12, 16], [13, 15]]

# Create the a single class of decision variables where
# From = {𝑫𝒂,𝑯𝒐} and To = {𝑪𝒉,𝑳𝑨,𝑺𝑭,𝑵𝒀}.
x = model.addVars(2, 4, lb=0, vtype=GRB.CONTINUOUS, name="Source Nodes")
# From = {𝑪𝒉,𝑳𝑨} and To = {𝑺𝑭,𝑵𝒀}.
y = model.addVars(2, 2, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment Nodes")

# The objective function
source_objective = gb.quicksum(source_costs[i][j]*x[i,j] for i in range(2) for j in range(4))
trans_objective = gb.quicksum(trans_costs[i][j]*y[i,j] for i in range(2) for j in range(2))
model.setObjective(source_objective + trans_objective, GRB.MINIMIZE)

# Add the supply constraints from source nodes
model.addConstr(gb.quicksum(x[0,j] for j in range(4)) <= 200, name="Supply Constraint 1")
model.addConstr(gb.quicksum(x[1,j] for j in range(4)) <= 160, name="Supply Constraint 2")
    
# Add the supply constraints from transshipment nodes
model.addConstr(gb.quicksum(x[i,0] for i in range(2)) <= 90, name="Transship Capacity 1")
model.addConstr(gb.quicksum(x[i,1] for i in range(2)) <= 80, name="Transship Capacity 2")

# Add the flow balance constrainits
model.addConstr(gb.quicksum(x[i,0] for i in range(2)) == gb.quicksum(y[0,k] for k in range(2)), name="Flow Balance 1")
model.addConstr(gb.quicksum(x[i,1] for i in range(2)) == gb.quicksum(y[1,k] for k in range(2)), name="Flow Balance 2")

# Add the demand constraints
model.addConstr(gb.quicksum(x[i,2] + y[i,0] for i in range(2)) == 140, name="Demand Constraint 1")
model.addConstr(gb.quicksum(x[i,3] + y[i,1] for i in range(2)) == 140, name="Demand Constraint 2")

# Ratio constraint
model.addConstr(0.6*gb.quicksum(y[i,j] for i in range(2) for j in range(2)) <= 0.4*gb.quicksum(x[i,j] for i in range(2) for j in range(2,4)), name="Ratio constraint")

# Optimally solve the problem
model.optimize()

# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

# Value of the objective function
print("Total Transportation cost: ", model.objVal)

# Print the decision variables
print(model.printAttr('X'))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 22.4.0 22E261)

CPU model: Intel(R) Core(TM) i5-1030NG7 CPU @ 1.10GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 12 columns and 36 nonzeros
Model fingerprint: 0x828d4180
Coefficient statistics:
  Matrix range     [4e-01, 1e+00]
  Objective range  [9e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 2e+02]
Presolve time: 0.01s
Presolved: 9 rows, 12 columns, 36 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7800000e+03   4.480000e+02   0.000000e+00      0s
       5    6.9040000e+03   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.904000000e+03
Number of Decision Variables:  12
Total Transportation cost:  6904.0

    Variable            X 
-------------------------
Source Nodes[0,1]           46 
Source Nodes[0,2]           74